## Module import 

In [1]:
#### 전처리
import pandas as pd
import numpy as np
# Scailing
from sklearn.preprocessing import PowerTransformer
#PCA
from sklearn.decomposition import PCA
#### Modeling
from pycaret.regression  import *

# 버전 확인
print('pandas : %s'%(pd.__version__))
print('numpy : %s'%(np.__version__))
print('pycaret : %s'%(pycaret.__version__))

!python --version

pandas : 1.3.2
numpy : 1.19.5
pycaret : 2.3.3
Python 3.8.8


## data 불러오기

In [2]:
# 데이터가 저장된 경로 설정
data_path = '../data/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'train_df.csv', encoding='cp949')
test = pd.read_csv(data_path + 'test_df.csv', encoding='cp949')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 전처리

In [3]:
# train과 test 병합
data = pd.concat([train,test]).reset_index(drop=True);data

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0
3,3,1154500002014200,5011000315087400,식품,농산물,10.0
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0
...,...,...,...,...,...,...
36635,36635,5013000858004400,4725000719072200,식품,농산물,NaN
36636,36636,5013000870018300,2826000106075300,식품,농산물,NaN
36637,36637,5013000897086300,4311100034004300,식품,농산물,NaN
36638,36638,5013000902065100,4145000013011200,식품,농산물,NaN


- 원본 데이터의 형태로 변경해줌

In [4]:
# data
data['SEND_SPG_INNB'] = data['SEND_SPG_INNB'].apply(lambda x: int(str(x)[:5])*10000000)
data['REC_SPG_INNB'] = data['REC_SPG_INNB'].apply(lambda x: int(str(x)[:5])*10000000)

In [5]:
from sklearn.cluster import KMeans

In [6]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, random_state=0)
kmeans.fit(data[['SEND_SPG_INNB', 'REC_SPG_INNB']])
data['cluster_kmeans'] = kmeans.labels_

In [7]:
# 범주형으로 취급
data[['SEND_SPG_INNB', 'REC_SPG_INNB', 'cluster_kmeans']] = data[['SEND_SPG_INNB', 'REC_SPG_INNB', 'cluster_kmeans']].astype(str)

- Make Feature

    - train

In [8]:
# 송하인 기준으로 총 몇개의 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_SEND_SPG_INNB_sum = data.groupby('SEND_SPG_INNB').sum()['INVC_CONT'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_SEND_SPG_INNB_sum_dix_tr = {x:y for x,y in zip(groupby_SEND_SPG_INNB_sum['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_sum['INVC_CONT'])}
# 피처 생성
data['총물류량'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_sum_dix_tr[x])

In [9]:
####data#####
# 송하인 기준으로 총 몇명의 수하인에게 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
groupby_SEND_SPG_INNB_count = data.groupby('SEND_SPG_INNB').count()['REC_SPG_INNB'].reset_index()
# 이후에 피처로 만들어주기 위한 dic생성
groupby_SEND_SPG_INNB_count_dix_tr = {x:y for x,y in zip(groupby_SEND_SPG_INNB_count['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_count['REC_SPG_INNB'])}
# 피처 생성
data['거래수하인수'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_count_dix_tr[x])

In [12]:
data['거래수하인수'][3200:].describe()

count    33440.000000
mean     11721.172578
std       5941.391698
min          1.000000
25%      13032.000000
50%      13032.000000
75%      16018.000000
max      16018.000000
Name: 거래수하인수, dtype: float64

In [10]:
# # train의 분포를 보고 test에 피처를 만들어줌(소분류를 보고 점수를 메김)
# # -> test또한 train의 분포를 따라갈것이라는 가정하에 train의 소분류별 INVC_CONT 평균을 내어 계산해줌
# tr_DL_GD_MCLS_NM = train.groupby('DL_GD_MCLS_NM')['INVC_CONT'].mean()
# tr_DL_GD_MCLS_NM = tr_DL_GD_MCLS_NM.reset_index()
# tr_DL_GD_MCLS_NM_dic = {x:y for x,y in zip(tr_DL_GD_MCLS_NM['DL_GD_MCLS_NM'], tr_DL_GD_MCLS_NM['INVC_CONT'])}
# data['예상점수'] = data['DL_GD_MCLS_NM'].apply(lambda x: tr_DL_GD_MCLS_NM_dic[x])

# tr_DL_GD_MCLS_NM_dic = {x:y for x,y in zip(tr_DL_GD_MCLS_NM['DL_GD_MCLS_NM'], tr_DL_GD_MCLS_NM['INVC_CONT'])}
# data['예상점수'] = data['DL_GD_MCLS_NM'].apply(lambda x: tr_DL_GD_MCLS_NM_dic[x])

In [11]:
# ####data#####

# # 송하인 기준으로 총 몇개의 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
# groupby_SEND_SPG_INNB_sum = data.groupby('SEND_SPG_INNB').sum()['예상점수'].reset_index()
# # 이후에 피처로 만들어주기 위한 dic생성
# groupby_SEND_SPG_INNB_sum_dix_tr = {x:y for x,y in zip(groupby_SEND_SPG_INNB_sum['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_sum['예상점수'])}
# # 피처 생성
# data['총물류량'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_sum_dix_tr[x])
# data = data.drop('예상점수', axis=1)

In [12]:
# ####data#####
# # 송하인 기준으로 총 몇명의 수하인에게 택배를 보냈는지 확인하여 이를 피처로 만들어줌 
# groupby_SEND_SPG_INNB_count = data.groupby('SEND_SPG_INNB').count()['REC_SPG_INNB'].reset_index()
# # 이후에 피처로 만들어주기 위한 dic생성
# groupby_SEND_SPG_INNB_count_dix_tr = {x:y for x,y in zip(groupby_SEND_SPG_INNB_count['SEND_SPG_INNB'], groupby_SEND_SPG_INNB_count['REC_SPG_INNB'])}
# # 피처 생성
# data['거래수하인수'] = data['SEND_SPG_INNB'].apply(lambda x: groupby_SEND_SPG_INNB_count_dix_tr[x])

In [13]:
####data####
# 평균 한건당 몇번의 수화물을 보내는지에 대한 피처 생성
data['평균거래량'] = data['총물류량'] / data['거래수하인수'] # INVC_CONT과의 상관계수를 확인해보면 scailing을 하기 전 0.6정도 나옴

    - preprocessing

In [14]:
####data####
# 인코딩과 스케일링을 위하여 분리및 제거해줌
data_index = data['index']
y_data = data['INVC_CONT']

data = data.drop(columns=['index', 'INVC_CONT'])

- Scailing

In [15]:
# 범주형 변수와 수치형 변수를 분리
#data
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 

In [16]:
# # 송하인과 수하인의 고유번호는 따로 scailing을 하지 않고 그대로 진행
# # data
# data_num_features = data_num_features[2:]

In [17]:
# #수치형변수 PowerTransformer scailing
# #data
# data[data_num_features] = PowerTransformer(standardize=True).fit_transform(data[data_num_features])

In [18]:
#위에서 떼어놨던 피처 병합
# data
data = pd.concat([data_index, data, y_data],axis=1)

### One-hot encoding & PCA
- 범주형 변수들을 onehot encoding한 이후 피처들의 차원을 두번에 걸쳐 줄여줌

In [19]:
data = pd.get_dummies(data, columns=['SEND_SPG_INNB', 'REC_SPG_INNB', 'cluster_kmeans'])

In [20]:
# 차원축소 매소드 
def dummy_to_pca(tr, column_name:str) :
    # pca에 쓰일 차원을 확인하기 위함
    max_d = len(set(tr[column_name])) # -> PCA의 차원으로 씀
    
    # PCA를 적용할 데이터프레임 생성
    f =tr[['index', column_name]] # 해당 컬럼만 가져와줌
    f = pd.get_dummies(f, columns=[column_name])  # 가져온 피처에 One-hot-encoding을 진행해줌
    f = f.iloc[:,1:].astype('float64') # 누적 분산 계산을 위해 index를 지우고 type을 바꿔줌
    
    # PCA_1
    pca = PCA(n_components=max_d) 
    pca.fit(f) # pca 적용 / 원핫 인코딩을 한 피처들을 위에서 정의한 차원을 축소해줌

    # 적절한 차원의 수를 찾기 위한 과정
    cumsum = np.cumsum(pca.explained_variance_ratio_) #분산의 설명량을 누적합 / 리스트의 형태
    num_d = np.argmax(cumsum >= 0.99) + 1 # 분산의 설명량이 99%이상 되는 인덱스를 cumsum에서 찾아줌 거기에 1을 더한것을 차원으로 지정 / 차원 갱신
    if num_d == 1: # 가장 처음의 차원이 분산이 제일 큰 경우
        num_d = max_d
    
    # PCA_2
    pca = PCA(n_components=num_d) # 분산의 설명량이 99% 이상 되는 차원으로 두번째 pca를 적용함    
    result = pca.fit_transform(f)
    result = pd.DataFrame(result)
    result.columns = [column_name + '_' + str(column) for column in result.columns] #열이름 지정
    result.index = f.index # 인덱스를 맞춰줌
    return result.reset_index()

In [21]:
# PCA를 진행한 데이터 생성후 data와 병합
# data
pca_data = pd.concat([dummy_to_pca(data, 'DL_GD_LCLS_NM').iloc[:,1:], dummy_to_pca(data, 'DL_GD_MCLS_NM').iloc[:,1:]],axis=1)
data = pd.concat([data.drop(columns=['DL_GD_LCLS_NM', 'DL_GD_MCLS_NM']), pca_data], axis=1)

- Data split

In [22]:
#train과 test를 분리
train = data.iloc[:32000,:]
test = data.iloc[32000:,:]
test = test.drop(columns = ['INVC_CONT']).reset_index(drop=True)

## Modeling with pycaret
- pycaret을 통하여 가장 좋은 성능을 내는 모델을 찾고 사용
- 검증 score만이 아닌 public을 같이 확인하여 모델 평가

- Model setting

In [23]:
train = train.iloc[:,1:]

In [24]:
reg = setup(train, 
            preprocess = False, 
            train_size = 0.8,  
            target = 'INVC_CONT', # 목표 변수
            silent = True, 
            use_gpu = True, 
            session_id = 42,
            fold_shuffle = True,
            )

,Description,Value
0,session_id,42
1,Target,INVC_CONT
2,Original Data,"(32000, 520)"
3,Missing Values,False
4,Numeric Features,519
5,Categorical Features,0
6,Transformed Train Set,"(25600, 519)"
7,Transformed Test Set,"(6400, 519)"
8,Shuffle Train-Test,True
9,Stratify Train-Test,False


- Compare Model -> 잘 나오는 모델이 무엇인지 확인하여 사용

In [25]:
top3 = compare_models(sort='RMSE', fold=5, n_select = 3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,2.1724,28.8443,5.3209,0.0748,0.4149,0.4198,1.0540
gbr,Gradient Boosting Regressor,2.1669,29.1442,5.3523,0.0629,0.4075,0.4175,12.0120
catboost,CatBoost Regressor,2.1577,29.6740,5.4057,0.0420,0.4106,0.4126,5.0600
ridge,Ridge Regression,2.2085,30.0185,5.4126,0.0501,0.4229,0.4285,0.2400
br,Bayesian Ridge,2.2003,30.1176,5.4131,0.0529,0.4199,0.4265,1.1520
omp,Orthogonal Matching Pursuit,2.1789,30.0275,5.4148,0.0491,0.4153,0.4206,0.1560
xgboost,Extreme Gradient Boosting,2.1599,29.7835,5.4183,0.0356,0.4119,0.4133,1.9060
lr,Linear Regression,2.2358,30.2868,5.4397,0.0397,0.4299,0.4365,0.4380
lasso,Lasso Regression,2.2469,31.9482,5.5589,0.0076,0.4206,0.4373,5.4140
en,Elastic Net,2.2469,31.9482,5.5589,0.0076,0.4206,0.4373,5.4460


- Hyperparameter tuning

In [26]:
#lightgbm
lightgbm = create_model('lightgbm')
#hyperparameter tuning
lightgbm = tune_model(lightgbm, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.1958,23.3830,4.8356,0.1336,0.4309,0.4304
1,2.2013,26.0488,5.1038,-0.0072,0.4307,0.4293
2,2.1839,26.9823,5.1945,-0.0010,0.4315,0.4241
3,2.2914,44.4576,6.6677,0.2321,0.4264,0.4132
4,2.1876,25.0564,5.0056,-0.0797,0.4274,0.4414
Mean,2.2120,29.1856,5.3614,0.0556,0.4294,0.4277
SD,0.0402,7.7285,0.6639,0.1119,0.0021,0.0092


In [27]:
#gbr
gbr = create_model('gbr')
#hyperparameter tuning
gbr = tune_model(gbr, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.1518,23.0236,4.7983,0.1469,0.4165,0.4216
1,2.1449,25.6186,5.0615,0.0095,0.4141,0.4160
2,2.1216,25.7414,5.0736,0.0451,0.4101,0.4099
3,2.2567,44.5174,6.6721,0.2310,0.4147,0.4026
4,2.1497,24.8486,4.9848,-0.0707,0.4119,0.4307
Mean,2.1649,28.7499,5.3181,0.0723,0.4135,0.4161
SD,0.0471,7.9433,0.6842,0.1057,0.0022,0.0096


In [28]:
#catboost
catboost = create_model('catboost')
#hyperparameter tuning
catboost = tune_model(catboost, optimize = 'RMSE', choose_better = True, fold = 5, n_iter = 30)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.1652,22.6445,4.7586,0.1610,0.4084,0.4262
1,2.1399,24.5475,4.9545,0.0509,0.4055,0.4152
2,2.1512,26.2037,5.1190,0.0279,0.4081,0.4217
3,2.2835,47.8234,6.9154,0.1739,0.4123,0.4108
4,2.1261,24.2572,4.9252,-0.0452,0.4026,0.4230
Mean,2.1732,29.0953,5.3345,0.0737,0.4074,0.4194
SD,0.0566,9.4319,0.7987,0.0830,0.0032,0.0056


In [34]:
models = []
for m in top3:
    models.append(tune_model(m,optimize = 'RMSE',choose_better = True,n_iter = 30))

In [30]:
# models = []
# models.append(lightgbm)
# models.append(gbr)
# models.append(catboost)

In [32]:
# bagging = ensemble_model(catboost, method = 'Bagging')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,2.1506,24.6836,4.9683,-0.1492,0.4027,0.4218
1,2.1856,23.4823,4.8459,0.2774,0.4142,0.4268
2,2.1062,22.4657,4.7398,0.0117,0.4040,0.4196
3,2.1841,27.4830,5.2424,0.0518,0.4096,0.4123
4,2.1925,29.5730,5.4381,0.0324,0.4126,0.4176
5,2.0832,22.5345,4.7471,0.0344,0.3994,0.4184
6,2.2588,39.9436,6.3201,0.2250,0.4174,0.4181
7,2.2505,46.4954,6.8188,0.2763,0.4051,0.4068
8,2.1970,33.8126,5.8149,-0.0950,0.4069,0.4182
9,2.0397,15.0800,3.8833,0.0290,0.3971,0.4215


In [33]:
# final_model = finalize_model(bagging)

In [35]:
# final_model = blend_models(estimator_list = models, optimize = 'RMSE')

In [24]:
# 전체 데이터로 재학습
final_model = finalize_model(lightgbm)

- Fit & predict

In [34]:
y_train = train['INVC_CONT']
train = train.drop('INVC_CONT', axis=1)

In [35]:
model = final_model
model.fit(train,y_train)
pred = model.predict(test.iloc[:,1:])

In [36]:
submission['INVC_CONT'] = pred

### Make submission 

In [37]:
t = pd.Timestamp.now()
fname = f'submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv'
# fname = 'submission_검증'
submission.to_csv('../submission/'+fname, index=False)
print("'{}' is ready to submit." .format(fname))

'submission_12211904.csv' is ready to submit.


In [40]:
# submission.to_csv('../submission/submission_검증.csv', index=False)
# print('submission is ready to submit.')

# END